In [1]:
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../../src")

In [2]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import slayerSNN as snn
from learningStats import learningStats
from IPython.display import HTML
import zipfile

In [3]:
netParams = snn.params('network.yaml')

In [4]:
# Dataset definition
class nmnistDataset(Dataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength):
        self.path = datasetPath 
        self.samples = np.loadtxt(sampleFile).astype('int')
        self.samplingTime = samplingTime
        self.nTimeBins    = int(sampleLength / samplingTime)

    def __getitem__(self, index):
        inputIndex  = self.samples[index, 0]
        classLabel  = self.samples[index, 1]

        inputSpikes = snn.io.readNpSpikes(
                        self.path + str(inputIndex.item()) + '.npy'
                        ).toSpikeTensor(torch.zeros((2,300,200,self.nTimeBins)),
                        samplingTime=self.samplingTime)
        desiredClass = torch.zeros((2, 1, 1, 1))
        desiredClass[classLabel,...] = 1
        print(str(inputIndex.item()))
        return inputSpikes, desiredClass, classLabel

    def __len__(self):
        return self.samples.shape[0]

In [5]:
class Network(torch.nn.Module):
    def __init__(self, netParams):
        super(Network, self).__init__()
        # initialize slayer
        slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        # define network functions
        self.conv1 = slayer.conv(1, 8, 4, stride=2, padding=1)
        self.conv2 = slayer.conv(8, 16, 4, stride=2, padding=1)
        self.conv3 = slayer.conv(16, 32, 4, stride=2, padding=1)
        self.pool1 = slayer.pool(2)
        self.pool2 = slayer.pool(2)
        self.fc1   = slayer.dense((4, 5, 32), 2)

    def forward(self, spikeInput):
        spikeLayer1 = self.slayer.spike(self.conv1(self.slayer.psp(spikeInput ))) # 32, 32, 16
        spikeLayer2 = self.slayer.spike(self.pool1(self.slayer.psp(spikeLayer1))) # 16, 16, 16
        spikeLayer3 = self.slayer.spike(self.conv2(self.slayer.psp(spikeLayer2))) # 16, 16, 32
        spikeLayer4 = self.slayer.spike(self.pool2(self.slayer.psp(spikeLayer3))) #  8,  8, 32
        spikeLayer5 = self.slayer.spike(self.conv3(self.slayer.psp(spikeLayer4))) #  8,  8, 64
        spikeOut    = self.slayer.spike(self.fc1  (self.slayer.psp(spikeLayer5))) #  10

        return spikeOut

In [6]:
# Define the cuda device to run the code on.
# device = torch.device('cuda')
# Use multiple GPU's if available
device = torch.device('cuda:0') # should be the first GPU of deviceIDs
deviceIds = [0, 1]

# Create network instance.
# net = Network(netParams).to(device)
# Split the network to run over multiple GPUs
net = torch.nn.DataParallel(Network(netParams).to(device), device_ids=deviceIds)

# Create snn loss instance.
error = snn.loss(netParams).to(device)

# Define optimizer module.
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)
    
# Dataset and dataLoader instances.
trainingSet = nmnistDataset(datasetPath ="/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_100ms_aug_3/",#"/home/student1/hyeongilee/slayerPytorch/example/dataset/KH/rain/raw_numpy_100ms/2020_07_30_18_20_29/", 
                            sampleFile  ="/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_100ms_aug_3/train.txt",#"/home/student1/hyeongilee/slayerPytorch/example/dataset/KH/rain/raw_numpy_100ms/2020_07_30_18_20_29/temp.txt",
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
trainLoader = DataLoader(dataset=trainingSet, batch_size=3, shuffle=False, num_workers=2)
trainingSet2 = nmnistDataset(datasetPath ="/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_100ms_aug_3/", 
                            sampleFile  ="/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_100ms_aug_3/test.txt",
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
trainLoader2 = DataLoader(dataset=trainingSet, batch_size=3, shuffle=False, num_workers=2)
trainingSet3 = nmnistDataset(datasetPath ="/home/student1/hyeongilee/slayerPytorch/example/dataset/KH/clean/aug_numpy_100ms/2020_07_04_11_08_58/0_5/", 
                            sampleFile  ="/home/student1/Downloads/dummy.txt",
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
trainLoader3 = DataLoader(dataset=trainingSet, batch_size=3, shuffle=False, num_workers=2)


# Learning stats instance.
stats = learningStats()#72553 8001 5277

FileNotFoundError: [Errno 2] No such file or directory: '/media/student1/770a08ae-0a7d-4b62-bbf4-b81fdd9baf2a/train_dataset_100ms_aug_3/train.txt'

In [ ]:
input, target, label = trainingSet[310]
print(label)
print(input.shape)
print(type(input))
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 300, 200, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

In [ ]:
input, target, label = trainingSet2[1128]
print(input.shape)
print(label)
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 300, 200, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

In [ ]:
input, target, label = trainingSet3[500]
print(target)
print(label)
print(input.shape)
print(type(input))
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 320, 320, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())